In [5]:
from dotenv import load_dotenv
load_dotenv()
import os
from groq import Groq
from pinecone import Pinecone, ServerlessSpec


In [15]:
pc=Pinecone(api_key=os.getenv("PINECONE-API-KEY"))
pc.create_index(
    name="rag",dimension=1024,metric="cosine",spec=ServerlessSpec(cloud="aws",region="us-east-1")
)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': '350c3a0705497432f80308f5e9d5e656', 'Date': 'Sun, 08 Sep 2024 16:40:39 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [8]:
import json
data=json.load(open("reviews.json"))
data['reviews']

[{'professor': 'Dr. John Smith',
  'subject': 'Mathematics',
  'stars': 5,
  'review': 'Dr. Smith is an excellent professor. His lectures are clear, and he always makes time for students.'},
 {'professor': 'Prof. Lisa Turner',
  'subject': 'Physics',
  'stars': 4,
  'review': 'Prof. Turner is knowledgeable and engaging, but her exams are quite challenging.'},
 {'professor': 'Dr. Michael Brown',
  'subject': 'Chemistry',
  'stars': 3,
  'review': "Dr. Brown's lectures can be a bit dry, but the lab sessions are well-organized."},
 {'professor': 'Prof. Emily Davis',
  'subject': 'Biology',
  'stars': 5,
  'review': 'Prof. Davis is passionate about her subject and encourages students to think critically.'},
 {'professor': 'Dr. Robert Wilson',
  'subject': 'Computer Science',
  'stars': 2,
  'review': "Dr. Wilson's lectures are difficult to follow, and he rarely responds to emails."},
 {'professor': 'Prof. Sarah Johnson',
  'subject': 'History',
  'stars': 4,
  'review': 'Prof. Johnson is w

In [14]:
processed_data= []
client= Groq()
for review in data['reviews']:
    response=client.embeddings.create(
        input=review['review'],
        model="text-embedding-ada-002",
    )
    embedding=response.data[0].embedding
    processed_data.append({
        "values":embedding,
        "id":review["professor"],
        "metadata":{
            "review":review["review"],
            "subject":review["subject"],
            "stars":review["stars"]

        }
    })


GroqError: The api_key client option must be set either by passing api_key to the client or by setting the GROQ_API_KEY environment variable

In [16]:
processed_data[0]

IndexError: list index out of range

In [17]:
index=pc.Index('rag')
index.upsert(
    vectors=processed_data,
    namespace="ns1"

)

PineconeApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Sun, 08 Sep 2024 16:47:13 GMT', 'Content-Type': 'application/json', 'Content-Length': '74', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '38', 'x-pinecone-request-id': '1580273311503660920', 'x-envoy-upstream-service-time': '38', 'server': 'envoy'})
HTTP response body: {"code":3,"message":"No vectors provided for upsert request","details":[]}


In [18]:
index.describe_index_stats()

{'dimension': 1024,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}